In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [3]:
path = '/Users/paulbolton/Library/CloudStorage/OneDrive-AmplifiGroupLimited/04_Learnings/02_Personal_Projects/03_Kaggle/Kaggle_learnings/Intro to Machine Learning/Data/'
os.listdir(path)

['melb_data.csv.zip',
 '.DS_Store',
 'test.csv',
 'melb_data.csv',
 'data_description.txt',
 'test (1).csv',
 'train (1).csv',
 'train.csv',
 'sample_submission.csv']

In [4]:
df_test = pd.read_csv(path+'test.csv')
df_train = pd.read_csv(path+'train.csv')
df_melbourne_data = pd.read_csv(path+'melb_data.csv')

In [45]:
df_test.shape

(1459, 80)

In [4]:
df_melbourne_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [5]:
df_train_desc = df_train.describe().copy()

In [6]:
list(df_train_desc.columns)

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

In [7]:
# What is the average lot size (rounded to nearest integer)?
df_train_desc['LotArea'].loc[df_train_desc.index == 'mean'].round()

mean    10517.0
Name: LotArea, dtype: float64

In [8]:
# As of today, how old is the newest home (current year - the date in which it was built)
2024 - df_train_desc['YearBuilt'].loc[df_train_desc.index == 'max'].round()

max    14.0
Name: YearBuilt, dtype: float64

In [9]:
# df_melbourne_data.columns.tolist()
df_melbourne_data = df_melbourne_data.dropna(axis=0)

In [10]:
y = df_melbourne_data.Price
features = ['Rooms','Bathroom','Landsize','Lattitude','Longtitude']
X = df_melbourne_data[features]

In [11]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [12]:
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


In [13]:
melbourne_model = DecisionTreeRegressor(random_state=1)
melbourne_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [14]:
df_melbourne_data[['Rooms','Bathroom','Landsize','Lattitude','Longtitude','Price']]

,Rooms,Bathroom,Landsize,Lattitude,Longtitude,Price
1,2,1.0,156.0,-37.80790,144.99340,1035000.0
2,3,2.0,134.0,-37.80930,144.99440,1465000.0
4,4,1.0,120.0,-37.80720,144.99410,1600000.0
6,3,2.0,245.0,-37.80240,144.99930,1876000.0
7,2,1.0,256.0,-37.80600,144.99540,1636000.0
...,...,...,...,...,...,...
12205,3,2.0,972.0,-37.51232,145.13282,601000.0
12206,3,1.0,179.0,-37.86558,144.90474,1050000.0
12207,1,1.0,0.0,-37.85588,144.89936,385000.0
12209,2,1.0,0.0,-37.85581,144.99025,560000.0


In [17]:
predicted_home_prices = melbourne_model.predict(X)

In [18]:
mean_absolute_error(y, predicted_home_prices)

1115.7467183128902

In [22]:
df_train = pd.read_csv(path+'train.csv')

In [29]:
# df_train.columns.tolist()

In [5]:
y = df_train.SalePrice
feature_columns = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = df_train[feature_columns]

In [6]:
iowa_model = DecisionTreeRegressor(random_state=1)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)
iowa_model.fit(train_X, train_y)
val_predictions = iowa_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

29652.931506849316


In [7]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [37]:
# candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# candidate_max_leaf_nodes = [50,75,100,125,150,175,200,225,250]
candidate_max_leaf_nodes = [75,80,85,90,95,100,105,110,115,120,125]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
_
for i in candidate_max_leaf_nodes:
    my_mae = get_mae(i, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(i, my_mae))

Max leaf nodes: 75  		 Mean Absolute Error:  27114
Max leaf nodes: 80  		 Mean Absolute Error:  27389
Max leaf nodes: 85  		 Mean Absolute Error:  27536
Max leaf nodes: 90  		 Mean Absolute Error:  27411
Max leaf nodes: 95  		 Mean Absolute Error:  27268
Max leaf nodes: 100  		 Mean Absolute Error:  27282
Max leaf nodes: 105  		 Mean Absolute Error:  27171
Max leaf nodes: 110  		 Mean Absolute Error:  27402
Max leaf nodes: 115  		 Mean Absolute Error:  27507
Max leaf nodes: 120  		 Mean Absolute Error:  27503
Max leaf nodes: 125  		 Mean Absolute Error:  27597


In [38]:

final_model = DecisionTreeRegressor(max_leaf_nodes=75, random_state=1)
final_model.fit(train_X, train_y)

DecisionTreeRegressor(max_leaf_nodes=75, random_state=1)

# Random Forest

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [43]:
forest_model = RandomForestRegressor(max_leaf_nodes=100,random_state=1)
forest_model.fit(train_X, train_y)
predict_y = forest_model.predict(val_X)
print(mean_absolute_error(val_y, predict_y))

22040.650602640035


In [44]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
predict_y = forest_model.predict(val_X)
print(mean_absolute_error(val_y, predict_y))

21857.15912981083


In [47]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(X,y)
X = df_test[feature_columns]
predict_y = forest_model.predict(X)
# print(mean_absolute_error(val_y, predict_y))

In [48]:
predict_y

array([122656.58, 156789.  , 182959.  , ..., 151283.01, 127878.  ,
       225959.8 ])

In [54]:
output = pd.DataFrame({'Id':df_test.Id,
 'SalesPrice':predict_y})
output.to_csv('submission.csv',index=False)

In [16]:
y = df_train.SalePrice
feature_columns = ['MSSubClass','LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','1stFlrSF',
                   '2ndFlrSF','LowQualFinSF','GrLivArea','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
                   'TotRmsAbvGrd','Fireplaces','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
                   'PoolArea','MiscVal','MoSold','YrSold']
X = df_train[feature_columns]
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
predict_y = forest_model.predict(val_X)
print(mean_absolute_error(val_y, predict_y))

17905.8978630137


In [17]:
forest_model.fit(X, y)
predict_y = forest_model.predict(X)
print(mean_absolute_error(y, predict_y))

7067.169788356164


In [18]:
X = df_test[feature_columns]
predict_y = forest_model.predict(X)

In [21]:
output = pd.DataFrame({'Id':df_test.Id,
 'SalePrice':predict_y})
output.to_csv('submission2.csv',index=False)

# Intermediate Course

In [25]:
X_full = pd.read_csv(path+'train.csv',index_col = 'Id')
X_test_full = pd.read_csv(path+'test.csv',index_col = 'Id')

In [26]:
# Obtain target and predictors
y = X_full.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy()
X_test = X_test_full[features].copy()

In [27]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [28]:
X_train.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
619,11694,2007,1828,0,2,3,9
871,6600,1962,894,0,1,2,5
93,13360,1921,964,0,1,2,5
818,13265,2002,1689,0,2,3,7
303,13704,2001,1541,0,2,3,6


In [29]:
# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [30]:
# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

Model 1 MAE: 24015
Model 2 MAE: 23740
Model 3 MAE: 23528
Model 4 MAE: 23996
Model 5 MAE: 23706


In [31]:
# Define a model
my_model = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0) # Your code here


In [33]:
# Fit the model to the training data
my_model.fit(X, y)

# Generate test predictions
preds_test = my_model.predict(X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission3.csv', index=False)

# Missing Values

In [34]:
X_full = pd.read_csv(path+'train.csv',index_col = 'Id')
X_test_full = pd.read_csv(path+'test.csv',index_col = 'Id')

In [40]:
X_full.loc[X_full.SalePrice.isna()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,


In [44]:
# dropping all the values that dont have a saleprice (predictor column)
X_full.dropna(axis=0,subset=['SalePrice'],inplace=True)
y = X_full.SalePrice
# remove SalePrice from the dataframe
X_full.drop(['SalePrice'],axis=1,inplace=True)

In [79]:
# Only using numerical columns, ie removes all string data types
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

In [50]:
# Taking the training data set and splitting it into a training componant and testing componant
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [54]:
print(X_train.shape)
missing_val_count_by_column = X_train.isnull().sum() # counts the missing values for all the columns
print(missing_val_count_by_column[missing_val_count_by_column>0])# filters out columns that dont have missing values

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results. This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Drop columns with missing values

In [61]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
17837.82570776256


## Imputation

In [62]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()

In [64]:
print(X_train.shape)
missing_val_count_by_column = X_train.isnull().sum() # counts the missing values for all the columns
print(missing_val_count_by_column[missing_val_count_by_column>0])# filters out columns that dont have missing values

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [72]:
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

These methods are used to center/feature scale of a given data. It basically helps to normalize the data within a particular range

For this, we use Z-score method.

Z = x - μ / σ 

We do this on the training set of data.

1.Fit(): Method calculates the parameters μ and σ and saves them as internal objects.

2.Transform(): Method using these calculated parameters apply the transformation to a particular dataset.

3.Fit_transform(): joins the fit() and transform() method for transformation of dataset.

Code snippet for Feature Scaling/Standardisation(after train_test_split).

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_transform(X_train)
sc.transform(X_test)

We apply the same(training set same two parameters μ and σ (values)) parameter transformation on our testing set.


In [73]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


Given that thre are so few missing values in the dataset, we'd expect imputation to perform better than dropping columns entirely. However, we see that dropping columns performs slightly better! While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset. That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method. For instance, consider the GarageYrBlt column (which indicates the year that the garage was built). It's likely that in some cases, a missing value could indicate a house that does not have a garage. Does it make more sense to fill in the median value along each column in this case? Or could we get better results by filling in the minimum value along each column? It's not quite clear what's best in this case, but perhaps we can rule out some options immediately - for instance, setting missing values in this column to 0 is likely to yield horrible results!

## Generate test predictions

In [74]:
final_X_train = X_full.select_dtypes(exclude=['object'])
final_X_valid = X_test_full.select_dtypes(exclude=['object'])

In [75]:
my_imputer = SimpleImputer(strategy='median')# using the most occuring number
final_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [76]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


In [83]:
final_X_test = pd.DataFrame(my_imputer.transform(X_test))
final_X_test.columns = X_test.columns
preds_test = model.predict(final_X_test)

In [84]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission4.csv', index=False)

### Score for this submission is 16619.08

# Categorical Variables

In [85]:
X_full = pd.read_csv(path+'train.csv',index_col = 'Id')
X_test_full = pd.read_csv(path+'test.csv',index_col = 'Id')

In [86]:
# dropping all the values that dont have a saleprice (predictor column)
X_full.dropna(axis=0,subset=['SalePrice'],inplace=True)
y = X_full.SalePrice
# remove SalePrice from the dataframe
X_full.drop(['SalePrice'],axis=1,inplace=True)

In [89]:
cols_with_missing = [col for col in X_full.columns if X_full[col].isnull().any()] 
X_full.drop(cols_with_missing, axis=1, inplace=True)
X_test_full.drop(cols_with_missing, axis=1, inplace=True)

In [91]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [92]:
# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [93]:
# Fill in the lines below: drop columns in training and validation data
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

In [95]:
score_dataset(drop_X_train, drop_X_valid, y_train, y_valid)

17837.82570776256

Before jumping into ordinal encoding, we'll investigate the dataset. Specifically, we'll look at the 'Condition2' column. The code cell below prints the unique entries in both the training and validation sets.

In [96]:
print("Unique values in 'Condition2' column in training data:", X_train['Condition2'].unique())
print("\nUnique values in 'Condition2' column in validation data:", X_valid['Condition2'].unique())

Unique values in 'Condition2' column in training data: ['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']

Unique values in 'Condition2' column in validation data: ['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


### Step 2: Ordinal encoding
#### Part A
If you now write code to:

fit an ordinal encoder to the training data, and then
use it to transform both the training and validation data,
you'll get an error. Can you see why this is the case? (You'll need to use the above output to answer this question.

Fitting an ordinal encoder to a column in the training data creates a corresponding integer-valued label for each unique value that appears in the training data. In the case that the validation data contains values that don't also appear in the training data, the encoder will throw an error, because these values won't have an integer assigned to them. Notice that the 'Condition2' column in the validation data contains the values 'RRAn' and 'RRNn', but these don't appear in the training data -- thus, if we try to use an ordinal encoder with scikit-learn, the code will throw an error.

This is a common problem that you'll encounter with real-world data, and there are many approaches to fixing this issue.  For instance, you can write a custom ordinal encoder to deal with new categories.  The simplest approach, however, is to drop the problematic categorical columns.  

Run the code cell below to save the problematic columns to a Python list `bad_label_cols`.  Likewise, columns that can be safely ordinal encoded are stored in `good_label_cols`.


In [98]:
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

In [104]:
for col in object_cols: 
    print(col)
    print(set(X_valid[col]))
    print(set(X_train[col]))
    print(set(X_valid[col]).issubset(set(X_train[col])))

MSZoning
{'RH', 'FV', 'C (all)', 'RL', 'RM'}
{'RH', 'FV', 'C (all)', 'RL', 'RM'}
True
Street
{'Pave', 'Grvl'}
{'Pave', 'Grvl'}
True
LotShape
{'IR3', 'IR2', 'IR1', 'Reg'}
{'IR3', 'IR2', 'IR1', 'Reg'}
True
LandContour
{'HLS', 'Bnk', 'Low', 'Lvl'}
{'HLS', 'Bnk', 'Low', 'Lvl'}
True
Utilities
{'AllPub'}
{'AllPub', 'NoSeWa'}
True
LotConfig
{'Inside', 'FR3', 'CulDSac', 'Corner', 'FR2'}
{'Inside', 'FR3', 'CulDSac', 'Corner', 'FR2'}
True
LandSlope
{'Mod', 'Gtl'}
{'Sev', 'Mod', 'Gtl'}
True
Neighborhood
{'NPkVill', 'BrDale', 'Veenker', 'SWISU', 'NoRidge', 'NWAmes', 'Crawfor', 'IDOTRR', 'OldTown', 'MeadowV', 'Edwards', 'Mitchel', 'Sawyer', 'NridgHt', 'NAmes', 'Blmngtn', 'StoneBr', 'Somerst', 'Gilbert', 'Timber', 'SawyerW', 'ClearCr', 'CollgCr', 'BrkSide'}
{'NPkVill', 'BrDale', 'Veenker', 'SWISU', 'NoRidge', 'NWAmes', 'Crawfor', 'IDOTRR', 'OldTown', 'MeadowV', 'Edwards', 'Mitchel', 'Sawyer', 'NridgHt', 'NAmes', 'Blmngtn', 'StoneBr', 'Somerst', 'Gilbert', 'Timber', 'SawyerW', 'ClearCr', 'CollgCr', '

In [107]:
# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]

In [111]:
set(object_cols)- set(good_label_cols)

{'Condition2', 'Functional', 'RoofMatl'}

In [112]:
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

In [113]:
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['RoofMatl', 'Functional', 'Condition2']


In [116]:
from sklearn.preprocessing import OrdinalEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

ordinal_encoder = OrdinalEncoder()
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

In [117]:
print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
17098.01649543379


In [121]:
print(col)

SaleCondition


In [122]:
X_train[col].nunique()

6

So far, you've tried two different approaches to dealing with categorical variables. And, you've seen that encoding categorical data yields better results than removing columns from the dataset.

Soon, you'll try one-hot encoding. Before then, there's one additional topic we need to cover. Begin by running the next code cell without changes.

In [118]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

## Step 3: Investigating cardinality
#### Part A
The output above shows, for each column with categorical data, the number of unique values in the column. For instance, the 'Street' column in the training data has two unique values: 'Grvl' and 'Pave', corresponding to a gravel road and a paved road, respectively.

We refer to the number of unique entries of a categorical variable as the cardinality of that categorical variable. For instance, the 'Street' variable has cardinality 2.

Use the output above to answer the questions below.

In [ ]:
# Fill in the line below: How many categorical variables in the training data
# have cardinality greater than 10?
high_cardinality_numcols = 3

# Fill in the line below: How many columns are needed to one-hot encode the 
# 'Neighborhood' variable in the training data?
num_cols_neighborhood = 25


To one-hot encode a variable, we need one column for each unique entry.

Part B
For large datasets with many rows, one-hot encoding can greatly expand the size of the dataset. For this reason, we typically will only one-hot encode columns with relatively low cardinality. Then, high cardinality columns can either be dropped from the dataset, or we can use ordinal encoding.

As an example, consider a dataset with 10,000 rows, and containing one categorical column with 100 unique entries.

If this column is replaced with the corresponding one-hot encoding, how many entries are added to the dataset?
If we instead replace the column with the ordinal encoding, how many entries are added?
Use your answers to fill in the lines below.

Hint: To calculate how many entries are added to the dataset through the one-hot encoding, begin by calculating how many entries are needed to encode the categorical variable (by multiplying the number of rows by the number of columns in the one-hot encoding). Then, to obtain how many entries are added to the dataset, subtract the number of entries in the original column.

In [120]:
# How many entries are added to the dataset by
# replacing the column with a one-hot encoding?
OH_entries_added = 1e4*100 - 1e4

# How many entries are added to the dataset by
# replacing the column with an ordinal encoding?
label_entries_added = 0

Next, you'll experiment with one-hot encoding. But, instead of encoding all of the categorical variables in the dataset, you'll only create a one-hot encoding for columns with cardinality less than 10.

Run the code cell below without changes to set low_cardinality_cols to a Python list containing the columns that will be one-hot encoded. Likewise, high_cardinality_cols contains a list of categorical columns that will be dropped from the dataset.

In [123]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Exterior2nd', 'Exterior1st', 'Neighborhood']


# Step 4: One-hot encoding
Use the next code cell to one-hot encode the data in X_train and X_valid. Set the preprocessed DataFrames to OH_X_train and OH_X_valid, respectively.

The full list of categorical columns in the dataset can be found in the Python list object_cols.
You should only one-hot encode the categorical columns in low_cardinality_cols. All other categorical columns should be dropped from the dataset.

In [142]:
from sklearn.preprocessing import OneHotEncoder

# Use as many lines of code as you need!
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

In [143]:
print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
17525.345719178084
